In [2]:
from collections import deque
import time
import unittest

'''
Clase Nodo que contiene la información de cada nodo en el grafo como su valor, 
sus vecinos, distancia desde el nodo inicial, padre y si ha sido visitado.
'''

class Nodo:

    # Constructor de la clase Nodo
    def __init__(self, id_nodo):         
        self.id = id_nodo                   # Identificador del nodo
        self.vecinos = []                   # Lista de nodos vecinos (conexiones)
        self.visitado = False               # Marca si el nodo ha sido visitado

    # Método para agregar un vecino con verificación para evitar duplicados
    def agregar_vecino(self, vecino):       
        if vecino.id not in self.vecinos:   
            self.vecinos.append(vecino.id)

'''
Clase Grafo no dirigido que contiene los nodos y métodos para agregar nodos, agregar aristas, 
y realizar búsquedas BFS y DFS.
'''

class Grafo:
    
    # Constructor de la clase Grafo
    def __init__(self):         
        self.nodos = {}         # Diccionario que almacenará como key el identificador del nodo y como value la referencia al objeto Nodo
        self.num_nodos = 0      # Contador de nodos en el grafo

    # Método para agregar un nuevo nodo al grafo
    def agregar_nodo(self, id_nodo):         
        if id_nodo not in self.nodos:
            self.num_nodos += 1                  # Suma uno al contador de nodos
            nuevo_nodo = Nodo(id_nodo)           # Crea una nueva instancia de la clase Nodo
            self.nodos[id_nodo] = nuevo_nodo     # Agrega el nuevo nodo al diccionario de nodos

    # Método para agregar una arista entre dos nodos
    def agregar_arista(self, nodo_origen, nodo_destino):                  
        if nodo_origen not in self.nodos:                                 # Se comprueba si el nodo origen y el nodo destino existen
            self.agregar_nodo(nodo_origen)                                # Si no existen, se crean          
        if nodo_destino not in self.nodos:
            self.agregar_nodo(nodo_destino)

        self.nodos[nodo_origen].agregar_vecino(self.nodos[nodo_destino])  # Se agregan los nodos como vecinos entre sí
        self.nodos[nodo_destino].agregar_vecino(self.nodos[nodo_origen])   

    # Método que implementa el algoritmo BFS y retorna distancias, padres, ruta, árbol y tiempo de ejecución
    def bfs(self, id_inicio, id_meta=None):                 
        if id_inicio not in self.nodos:                         # Verifica si el nodo de inicio existe, caso contrario, retorna un mensaje de error
            return "Inicio no encontrado", None, None, 0

        start_time = time.perf_counter()         # Marca el tiempo de inicio de la búsqueda
        search_queue = deque()                   # Inicializa la cola para BFS
        
        # Inicializa los valores de los nodos
        for obj_nodo in self.nodos.values():     
            obj_nodo.visitado = False

        nodo_inicio = self.nodos[id_inicio]      # Se guarda la referencia al nodo inicial en una variable
        search_queue += [nodo_inicio]            # Se agrega a la cola
        nodo_inicio.visitado = True              # Se marca como visitado

        padres = {id_inicio: None}               # Se crea un diccionario para almacenar los padres de cada nodo
        distancias = {id_inicio: 0}              # Se crea un diccionario para almacenar las distancias desde el nodo inicial
        
        
        while search_queue:                      
            nodo = search_queue.popleft()                           # Se saca un nodo por la izquierda de la cola
            
            # Se recorren los vecinos del nodo actual
            for id_vecino in nodo.vecinos:                          
                vecino = self.nodos[id_vecino]                      # Se guarda la referencia al nodo vecino en una variable
                if not vecino.visitado:                             # Se comprueba si el vecino no ha sido visitado
                    search_queue += [vecino]                        # Se agrega el vecino a la cola
                    vecino.visitado = True                          # Se marca como visitado
                    padres[vecino.id] = nodo.id                     # Se actualiza el diccionario de padres
                    distancias[vecino.id] = distancias[nodo.id] + 1 # Se actualiza el diccionario de distancias
                    
                    if id_meta and vecino.id == id_meta:            # Verifica si se proporcionó un nodo meta y si se ha alcanzado, finaliza la ejecución y retorna los resultados.
                        end_time = time.perf_counter()              # Marca el tiempo de finalización de la búsqueda
                        return distancias, padres, self.reconstruir_ruta(id_inicio, id_meta, padres), end_time - start_time
        
        # Si no se proporciona un nodo meta o no se alcanzó, retorna los resultados al finalizar la búsqueda completa.
        end_time = time.perf_counter()                          # Marca el tiempo de finalización de la búsqueda
        return distancias, padres, None, end_time - start_time  # Retorna los diccionarios de distancias, padres, None (si no se encontró ruta) y el tiempo de ejecución
    
    # Método que implementa el algoritmo DFS de forma recursiva y retorna padres, árbol, ruta encontrada y tiempo de ejecución
    def dfs(self, id_inicio, id_meta=None):
    
        if id_inicio not in self.nodos:                         # Verifica si el nodo de inicio existe, caso contrario, retorna un mensaje de error
            return "Inicio no encontrado", None, None, None, 0

        start_time = time.perf_counter()   # Marca el tiempo de inicio de la búsqueda

        # Inicializa los valores de los nodos
        for obj_nodo in self.nodos.values():   
            obj_nodo.visitado = False
            
        padres = {id_inicio: None}          # Se crea un diccionario para almacenar los padres de cada nodo                                                

        ruta_encontrada = None              # Variable para almacenar la ruta encontrada (si se proporciona una meta)
        
        # Búsqueda DFS recursiva
        def dfs_visita(nodo_actual):
            nonlocal ruta_encontrada        # Utiliza nonlocal para modificar la variable externa
            nodo_actual.visitado = True     # Marca el nodo actual como visitado
            
            # Se recorren los vecinos del nodo actual
            for id_vecino in nodo_actual.vecinos:
                vecino = self.nodos[id_vecino]             # Se guarda la referencia al nodo vecino en una variable
                if not vecino.visitado:                    # Se comprueba si el vecino no ha sido visitado
                    # Se asigna el padre del vecino (dinámicamente en el objeto Nodo)
                    setattr(vecino, 'padre', nodo_actual)  # asigna atributo dinamicamente
                    padres[vecino.id] = nodo_actual.id     # Se actualiza el diccionario de padres
                    
                    if id_meta and vecino.id == id_meta:   # Verifica si se proporcionó un nodo meta y si se ha alcanzado, almacena la ruta encontrada.
                        ruta_encontrada = self.reconstruir_ruta(id_inicio, id_meta, padres)
                        
                    dfs_visita(vecino)                     # Llamada recursiva para visitar el vecino
                    
        dfs_visita(self.nodos[id_inicio])                  # Inicia la búsqueda DFS desde el nodo inicial

        # NO desenvuelvo el resultado: dejo la tupla (ruta, profundidad) para que los tests puedan desempacarla.
        camino_profundo = self.buscar_camino_mas_profundo(id_inicio, padres) # Se calcula el camino más profundo después de la búsqueda DFS
        
        end_time = time.perf_counter()                                                 # Marca el tiempo de finalización de la búsqueda
        return padres, ruta_encontrada, camino_profundo, end_time - start_time  # Retorna el diccionario de padres, la ruta encontrada (si existe), la tupla (ruta, profundidad) y el tiempo de ejecución

    # Método auxiliar para reconstruir la ruta desde el nodo meta hasta el nodo inicial usando el diccionario de padres
    def reconstruir_ruta(self, id_inicio, id_meta, padres):
        ruta = []                             # Lista para almacenar la ruta resultante
        actual = id_meta                      # Comienza desde el nodo meta
        while actual:                 
            ruta.append(actual)               # Agrega el nodo actual a la ruta
            actual = padres.get(actual)       # El padre del nodo actual se convierte en el nuevo nodo actual
            if actual == id_inicio:           # Si se llega al nodo inicio, se agrega y se termina la construcción de la ruta
                ruta.append(id_inicio)
                break
        
        # Si la meta es alcanzada, la ruta se construye al revés
        if ruta and ruta[-1] == id_inicio:
            return " -> ".join(ruta[::-1])  
        return None                           # Si no se alcanza la meta, retorna None

    # Método auxiliar para encontrar el camino más profundo desde el nodo inicial usando el diccionario de padres
    def buscar_camino_mas_profundo(self, id_inicio, padres):
        nodo_mas_profundo = id_inicio   # Inicializa con el nodo de inicio
        max_profundidad = 0             # Variable para almacenar la profundidad máxima encontrada
        
        # Se recorre cada nodo en el grafo para calcular su profundidad
        for nodo_id in self.nodos:
            profundidad_actual = 0           # Se actualiza la profundidad para el nodo actual
            actual = nodo_id                 # Se almacena el nodo actual
            
            temp_padre = padres.get(actual)  # Se obtiene el padre del nodo actual
            
            # Se recorre hacia el padre hasta llegar al inicio
            while temp_padre:
                profundidad_actual += 1             # Se incrementa la profundidad
                actual = temp_padre                 # El padre del nodo actual se convierte en el nuevo nodo actual
                temp_padre = padres.get(actual)     # Se obtiene el padre del nuevo nodo actual
            
            # Se verifica si se ha encontrado una mayor profundidad que la máxima registrada y se actualiza
            if actual == id_inicio and profundidad_actual > max_profundidad:
                max_profundidad = profundidad_actual
                nodo_mas_profundo = nodo_id
        
        # Se reconstruye la ruta desde el nodo más profundo hasta el inicio
        if nodo_mas_profundo:
            return self.reconstruir_ruta(id_inicio, nodo_mas_profundo, padres), max_profundidad
        return None, 0                  # Si no se encuentra ningún camino, retorna (None, 0)

    # Método para imprimir el árbol de búsqueda (BFS o DFS) en el terminal
    def imprimir_arbol(self, id_inicio, padres, algoritmo):

        print(f"\n--- ÁRBOL DE BÚSQUEDA EN {algoritmo} ---")
        
        # Se invierte el diccionario de padres a diccionario de hijos
        hijos = {}
        for hijo, padre in padres.items():  # Se recorre el diccionario de padres
            if padre is not None:           # Evita agregar el nodo raíz que no tiene padre
                if padre not in hijos:      # Inicializa la lista de hijos si el padre no está en el diccionario
                    hijos[padre] = []       
                hijos[padre].append(hijo)   # Se agrega el hijo a la lista del padre

        # Función recursiva para imprimir la estructura del árbol
        def imprimir_recursivo(nodo_id, prefijo=''):
            
            # Obtener los hijos y ordenarlos alfabéticamente para una salida estable
            lista_hijos = sorted(hijos.get(nodo_id, [])) # Obtiene la lista de hijos del nodo actual
            num_hijos = len(lista_hijos)                 # Se calcula el número de hijos
            
            for i, hijo_id in enumerate(lista_hijos):    # Se recorre cada hijo del nodo actual
                es_el_ultimo = (i == num_hijos - 1)      # Verifica si es el último hijo
                
                simbolo_conexion = "└── " if es_el_ultimo else "├── " # Se define el símbolo de conexión según si es el último hijo o no
                
                nuevo_prefijo_rama = prefijo + ("    " if es_el_ultimo else "│   ") # Se actualiza el prefijo para la siguiente rama
                
                print(prefijo + simbolo_conexion + hijo_id) # Se imprime el hijo con el prefijo y símbolo correspondiente
                imprimir_recursivo(hijo_id, nuevo_prefijo_rama) # Llamada recursiva para imprimir los hijos del hijo actual

        print(id_inicio)              # Se imprime la raíz del árbol
        imprimir_recursivo(id_inicio) # Se inicia la impresión recursiva desde el nodo inicial


# ===========================================================
# EJEMPLO DE EJECUCIÓN 
# ===========================================================
def ejemplo_ejecucion():
    g = Grafo()
    
    g.agregar_arista('A101', 'Pasillo_A') 
    g.agregar_arista('Pasillo_A', 'B205')
    g.agregar_arista('Pasillo_A', 'Salida_E1')
    g.agregar_arista('Pasillo_A', 'Patio_Central')
    g.agregar_arista('Patio_Central', 'Biblioteca')
    g.agregar_arista('Patio_Central', 'Salida_E2')
    g.agregar_arista('Biblioteca', 'Salida_E2')
    g.agregar_arista('B205', 'Laboratorio')
    
    # 1. PRUEBA BFS
    print("\n--- PRUEBA BFS (Ruta más corta / Evacuación Rápida) ---")
    inicio_bfs = 'A101'
    meta_bfs = 'Salida_E2'
    distancias, padres_bfs, ruta, tiempo = g.bfs(inicio_bfs, meta_bfs)
    
    if ruta:
        print(f"Ruta más corta de {inicio_bfs} a {meta_bfs}: {ruta}")
        print(f"Distancia (Número de nodos): {distancias.get(meta_bfs)}")
        print(f"Árbol BFS (Padres): {padres_bfs}")
        print(f"Tiempo de ejecución: {tiempo:.10f} segundos")
        
        # LLAMADA A LA VISUALIZACIÓN BFS
        g.imprimir_arbol(inicio_bfs, padres_bfs, "AMPLITUD (BFS Tree)")
    else:
        print(f"No se encontró ruta de {inicio_bfs} a {meta_bfs}")

    # 2. PRUEBA DFS
    print("\n--- PRUEBA DFS (Exploración / Verificación de Cobertura) ---")
    inicio_dfs = 'A101'
    meta_dfs_opcional = 'Laboratorio'
    padres_dfs, ruta_dfs_encontrada, camino_profundo, tiempo_dfs = g.dfs(inicio_dfs, meta_dfs_opcional)
    
    print(f"Árbol DFS (Padres de exploración): {padres_dfs}")
    print(f"Ruta DFS a la Meta '{meta_dfs_opcional}' (si existe): {ruta_dfs_encontrada}")
    print(f"Camino de Mayor Profundidad desde {inicio_dfs}: {camino_profundo}")
    print(f"Tiempo de ejecución: {tiempo_dfs:.10f} segundos")

    # LLAMADA A LA VISUALIZACIÓN DFS
    g.imprimir_arbol(inicio_dfs, padres_dfs, "PROFUNDIDAD (DFS Tree)")


# ===========================================================
# ANÁLISIS Y VALIDACIÓN COMPLETA – SISTEMA DE GRAFOS BFS/DFS
# ===========================================================

class AnalisisComplejidad:
    """Análisis teórico de complejidad de los algoritmos BFS y DFS."""

    @staticmethod
    def analizar_bfs():
        """
        COMPLEJIDAD TEMPORAL: O(V + E)
            - V: número de vértices
            - E: número de aristas

        Detalles:
            • Inicialización: O(V)
            • Procesamiento de cola: O(V + E)
            • Reconstrucción de ruta: O(V)

        COMPLEJIDAD ESPACIAL: O(V)
        """
        return {
            'Algoritmo': 'BFS (Breadth-First Search)',
            'Complejidad Temporal': 'O(V + E)',
            'Complejidad Espacial': 'O(V)',
            'Aplicación': 'Obtención de la ruta más corta en grafos no ponderados'
        }

    @staticmethod
    def analizar_dfs():
        """
        COMPLEJIDAD TEMPORAL: O(V + E)
            • Cada vértice se visita una vez.
            • Cada arista se explora una vez.

        COMPLEJIDAD ESPACIAL: O(V)
            • Pila de recursión en peor caso.
        """
        return {
            'Algoritmo': 'DFS (Depth-First Search)',
            'Complejidad Temporal': 'O(V + E)',
            'Complejidad Espacial': 'O(V)',
            'Aplicación': 'Exploración completa del grafo y detección de ciclos'
        }

    @staticmethod
    def comparar_algoritmos():
        """Comparación entre BFS y DFS."""
        return {
            'BFS': {
                'Ventajas': [
                    'Encuentra la ruta más corta',
                    'Adecuado para grafos no ponderados',
                    'Evita exploraciones profundas innecesarias'
                ],
                'Desventajas': [
                    'Mayor uso de memoria en grafos anchos'
                ]
            },
            'DFS': {
                'Ventajas': [
                    'Menor consumo de memoria en grafos profundos',
                    'Útil en problemas de backtracking'
                ],
                'Desventajas': [
                    'No garantiza ruta mínima',
                    'Puede caer en ciclos si no se controla'
                ]
            }
        }

# ===========================================================
# PRUEBAS UNITARIAS ADAPTADAS A LA IMPLEMENTACIÓN
# ===========================================================
class TestGrafoBasico(unittest.TestCase):
    """Pruebas unitarias de funcionalidades básicas del grafo."""

    def setUp(self):
        self.grafo = Grafo()

    def test_agregar_vertice(self):
        # Se adapta a la API real: agregar_nodo, nodos y num_nodos
        self.grafo.agregar_nodo('A')
        self.grafo.agregar_nodo('B')

        self.assertIn('A', self.grafo.nodos)
        self.assertIn('B', self.grafo.nodos)
        self.assertEqual(self.grafo.num_nodos, 2)

    def test_agregar_arista(self):
        self.grafo.agregar_arista('A', 'B')

        self.assertIn('A', self.grafo.nodos)
        self.assertIn('B', self.grafo.nodos)
        self.assertIn('B', self.grafo.nodos['A'].vecinos)
        self.assertIn('A', self.grafo.nodos['B'].vecinos)

    def test_grafo_vacio(self):
        resultado = self.grafo.bfs('A')
        self.assertEqual(resultado[0], "Inicio no encontrado")

    def test_un_solo_nodo(self):
        self.grafo.agregar_nodo('A')
        distancias, padres, ruta, _ = self.grafo.bfs('A')

        self.assertEqual(distancias, {'A': 0})
        self.assertEqual(padres, {'A': None})
        self.assertIsNone(ruta)

# ===========================================================
# PRUEBAS DE BFS
# ===========================================================
class TestBFS(unittest.TestCase):
    """Pruebas del algoritmo BFS."""

    def setUp(self):
        self.grafo = Grafo()

    def test_bfs_ruta_minima(self):
        self.grafo.agregar_arista('A', 'B')
        self.grafo.agregar_arista('B', 'C')

        distancias, padres, ruta, _ = self.grafo.bfs('A', 'C')

        self.assertEqual(ruta, "A -> B -> C")
        self.assertEqual(distancias['C'], 2)
        self.assertEqual(padres['C'], 'B')

    def test_bfs_multiple_caminos(self):
        self.grafo.agregar_arista('A', 'B')
        self.grafo.agregar_arista('A', 'C')
        self.grafo.agregar_arista('B', 'D')
        self.grafo.agregar_arista('C', 'E')
        self.grafo.agregar_arista('D', 'F')
        self.grafo.agregar_arista('E', 'F')

        distancias, _, _, _ = self.grafo.bfs('A', 'F')

        self.assertEqual(distancias['F'], 3)

    def test_bfs_sin_meta(self):
        self.grafo.agregar_arista('A', 'B')
        self.grafo.agregar_arista('B', 'C')
        self.grafo.agregar_arista('A', 'D')

        distancias, _, _, _ = self.grafo.bfs('A')

        self.assertEqual(set(distancias.keys()), {'A', 'B', 'C', 'D'})
        self.assertEqual(distancias['C'], 2)

# ===========================================================
# PRUEBAS DE DFS
# ===========================================================
class TestDFS(unittest.TestCase):
    """Pruebas del algoritmo DFS."""

    def setUp(self):
        self.grafo = Grafo()

    def test_dfs_exploracion_completa(self):
        self.grafo.agregar_arista('A', 'B')
        self.grafo.agregar_arista('B', 'C')
        self.grafo.agregar_arista('A', 'D')

        padres, _, _, _ = self.grafo.dfs('A')

        self.assertEqual(set(padres.keys()), {'A', 'B', 'C', 'D'})

    def test_dfs_camino_profundo(self):
        self.grafo.agregar_arista('A', 'B')
        self.grafo.agregar_arista('B', 'C')
        self.grafo.agregar_arista('C', 'D')
        self.grafo.agregar_arista('A', 'E')

        _, _, camino_profundo, _ = self.grafo.dfs('A')

        ruta_profunda, profundidad = camino_profundo
        self.assertEqual(ruta_profunda, "A -> B -> C -> D")
        self.assertEqual(profundidad, 3)

    def test_dfs_con_meta(self):
        self.grafo.agregar_arista('A', 'B')
        self.grafo.agregar_arista('B', 'C')
        self.grafo.agregar_arista('A', 'D')

        _, ruta, _, _ = self.grafo.dfs('A', 'C')

        self.assertIsNotNone(ruta)
        self.assertIn('C', ruta)

# ===========================================================
# PRUEBAS COMPARATIVAS
# ===========================================================
class TestComparativoBFSvsDFS(unittest.TestCase):
    """Comparación entre BFS y DFS en rutas."""

    def test_bfs_vs_dfs_diferencias(self):
        grafo = Grafo()
        grafo.agregar_arista('A', 'B')
        grafo.agregar_arista('A', 'C')
        grafo.agregar_arista('B', 'D')
        grafo.agregar_arista('C', 'D')
        grafo.agregar_arista('D', 'E')

        dist_bfs, _, ruta_bfs, _ = grafo.bfs('A', 'E')
        _, ruta_dfs, _, _ = grafo.dfs('A', 'E')

        self.assertEqual(dist_bfs['E'], 3)

# ===========================================================
# PRUEBAS DE ERRORES Y CASOS EXTREMOS
# ===========================================================
class TestManejoErrores(unittest.TestCase):
    """Pruebas para casos extremos y manejo de errores."""

    def setUp(self):
        self.grafo = Grafo()

    def test_nodos_inexistentes(self):
        self.assertEqual(self.grafo.bfs('X')[0], "Inicio no encontrado")
        self.assertEqual(self.grafo.dfs('X')[0], "Inicio no encontrado")

    def test_destino_inalcanzable(self):
        self.grafo.agregar_arista('A', 'B')
        self.grafo.agregar_arista('C', 'D')

        distancias, _, ruta, _ = self.grafo.bfs('A', 'D')

        self.assertIsNone(ruta)
        self.assertNotIn('D', distancias)

    def test_grafo_ciclico(self):
        self.grafo.agregar_arista('A', 'B')
        self.grafo.agregar_arista('B', 'C')
        self.grafo.agregar_arista('C', 'F')
        self.grafo.agregar_arista('F', 'D')
        self.grafo.agregar_arista('D', 'E')
        self.grafo.agregar_arista('E', 'A')

        distancias_bfs, _, ruta_bfs, _ = self.grafo.bfs('A', 'F')
        _, ruta_dfs, _, _ = self.grafo.dfs('A', 'F')

        self.assertIsNotNone(ruta_bfs)
        self.assertIsNotNone(ruta_dfs)
        self.assertLessEqual(distancias_bfs['F'], 3)

    def test_grafo_grande(self):
        grafo_grande = Grafo()
        n = 100

        for i in range(n - 1):
            grafo_grande.agregar_arista(str(i), str(i + 1))

        t1 = time.time()
        grafo_grande.bfs('0', str(n - 1))
        tiempo_bfs = time.time() - t1

        t2 = time.time()
        grafo_grande.dfs('0', str(n - 1))
        tiempo_dfs = time.time() - t2

        # Dependiendo del entorno, estas aserciones pueden fallar si el sistema está muy cargado.
        # Se mantienen aquí como comprobación de rendimiento razonable.
        self.assertLess(tiempo_bfs, 1.0)
        self.assertLess(tiempo_dfs, 1.0)

# ===========================================================
# ESCENARIO REAL: EVACUACIÓN
# ===========================================================
class TestEscenarioEvacuacion(unittest.TestCase):
    """Pruebas de rutas en un escenario simulado de evacuación."""

    def setUp(self):
        self.grafo = Grafo()
        self.grafo.agregar_arista('A101', 'Pasillo_A')
        self.grafo.agregar_arista('Pasillo_A', 'B205')
        self.grafo.agregar_arista('Pasillo_A', 'Salida_E1')
        self.grafo.agregar_arista('Pasillo_A', 'Patio_Central')
        self.grafo.agregar_arista('Patio_Central', 'Biblioteca')
        self.grafo.agregar_arista('Patio_Central', 'Salida_E2')
        self.grafo.agregar_arista('Biblioteca', 'Salida_E2')
        self.grafo.agregar_arista('B205', 'Laboratorio')

    def test_ruta_evacuacion_rapida(self):
        distancias, _, ruta, _ = self.grafo.bfs('A101', 'Salida_E2')

        # CORRECCIÓN: la ruta más corta con el grafo dado pasa por Patio_Central (3 saltos)
        self.assertEqual(ruta, "A101 -> Pasillo_A -> Patio_Central -> Salida_E2")
        self.assertEqual(distancias['Salida_E2'], 3)

    def test_multiples_salidas(self):
        for salida in ['Salida_E1', 'Salida_E2']:
            distancias, _, ruta, _ = self.grafo.bfs('A101', salida)
            self.assertIsNotNone(ruta)

    def test_evacuacion_desde_varios_puntos(self):
        ubicaciones = ['A101', 'B205', 'Laboratorio', 'Biblioteca']

        for u in ubicaciones:
            distancias, _, ruta, _ = self.grafo.bfs(u, 'Salida_E1')
            self.assertIsNotNone(ruta)

    def test_cobertura_completa_dfs(self):
        padres, _, camino_profundo, _ = self.grafo.dfs('A101')

        esperado = {
            'A101', 'Pasillo_A', 'B205', 'Salida_E1', 'Patio_Central',
            'Biblioteca', 'Salida_E2', 'Laboratorio'
        }

        self.assertEqual(set(padres.keys()), esperado)

# ===========================================================
# EJECUCIÓN PRINCIPAL: EJEMPLO + PRUEBAS UNITARIAS
# ===========================================================
if __name__ == '__main__':
    # Ejecutar ejemplo de uso primero (manteniendo la originalidad del ejemplo proporcionado)
    ejemplo_ejecucion()

    # Ejecutar pruebas unitarias
    print("\n\n--- INICIANDO PRUEBAS UNITARIAS ---")
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)


test_bfs_multiple_caminos (__main__.TestBFS.test_bfs_multiple_caminos) ... ok
test_bfs_ruta_minima (__main__.TestBFS.test_bfs_ruta_minima) ... ok
test_bfs_sin_meta (__main__.TestBFS.test_bfs_sin_meta) ... ok
test_bfs_vs_dfs_diferencias (__main__.TestComparativoBFSvsDFS.test_bfs_vs_dfs_diferencias) ... ok
test_dfs_camino_profundo (__main__.TestDFS.test_dfs_camino_profundo) ... ok
test_dfs_con_meta (__main__.TestDFS.test_dfs_con_meta) ... ok
test_dfs_exploracion_completa (__main__.TestDFS.test_dfs_exploracion_completa) ... ok
test_cobertura_completa_dfs (__main__.TestEscenarioEvacuacion.test_cobertura_completa_dfs) ... ok
test_evacuacion_desde_varios_puntos (__main__.TestEscenarioEvacuacion.test_evacuacion_desde_varios_puntos) ... ok
test_multiples_salidas (__main__.TestEscenarioEvacuacion.test_multiples_salidas) ... ok
test_ruta_evacuacion_rapida (__main__.TestEscenarioEvacuacion.test_ruta_evacuacion_rapida) ... ok
test_agregar_arista (__main__.TestGrafoBasico.test_agregar_arista) ... o


--- PRUEBA BFS (Ruta más corta / Evacuación Rápida) ---
Ruta más corta de A101 a Salida_E2: A101 -> Pasillo_A -> Patio_Central -> Salida_E2
Distancia (Número de nodos): 3
Árbol BFS (Padres): {'A101': None, 'Pasillo_A': 'A101', 'B205': 'Pasillo_A', 'Salida_E1': 'Pasillo_A', 'Patio_Central': 'Pasillo_A', 'Laboratorio': 'B205', 'Biblioteca': 'Patio_Central', 'Salida_E2': 'Patio_Central'}
Tiempo de ejecución: 0.0000223999 segundos

--- ÁRBOL DE BÚSQUEDA EN AMPLITUD (BFS Tree) ---
A101
└── Pasillo_A
    ├── B205
    │   └── Laboratorio
    ├── Patio_Central
    │   ├── Biblioteca
    │   └── Salida_E2
    └── Salida_E1

--- PRUEBA DFS (Exploración / Verificación de Cobertura) ---
Árbol DFS (Padres de exploración): {'A101': None, 'Pasillo_A': 'A101', 'B205': 'Pasillo_A', 'Laboratorio': 'B205', 'Salida_E1': 'Pasillo_A', 'Patio_Central': 'Pasillo_A', 'Biblioteca': 'Patio_Central', 'Salida_E2': 'Biblioteca'}
Ruta DFS a la Meta 'Laboratorio' (si existe): A101 -> Pasillo_A -> B205 -> Laboratorio